# Język Python - Laboratorium 3.

## Parametry z linii poleceń

### Zadanie

Dany jest program z dziedziny uczenia maszynowego. Należy napisać kod, który wczyta z linii poleceń następujące parametry:
- dane treningowe
- dane testowe
- opcjonalnie plik wyjściowy (domyślnie wyniki będą wypisywane na ekran)
- opcjonalnie plik z modelem (przyjmujemy, że jest jakiś plik, którego używamy domyślnie)
- algorytm (do wyboru spośród RNN, LSTM i CNN)
- ile przykładów negatywnych należy wygenerować (dowolna nieujemna liczba całkowita)
- rozmiar warstwy ukrytej (do wyboru spośród 100, 150 i 300)

python ml.py --algorithm RNN --negative 10 --hidden 100 train.csv test.csv

### sys.argv
- porównywanie stringów (operator `==`)
- operator `in`
- metoda `index`

### argparse

- pobiera i analizuje argumenty wywołania
- zastępuje przestarzały moduł optparse
- alternatywa dla getopt (vide: C) oraz ręcznego analizowania zmiennej sys.argv
- zarządza:
    - parametrami pozycyjnymi
    - parametrami opcjonalnymi (flagami) w wersji krótkiej i długiej
    - tekstem pomocy informującym o prawidłowym użyciu (w reakcji na opcję -h/--help)
    - komunikatem o błędnym wywołaniu

In [ ]:
import sys # potrzebne dla ręcznej modyfikacji sys.argv; w przypadku uruchamiania z konsoli można usunąć
import argparse

# nie róbcie tego w domu
def exit(*args, **kwargs):
    print("Program exited with {} and {}".format(args, kwargs))
    
sys.exit = exit

In [ ]:
sys.argv = ['script.py','-h']
parser = argparse.ArgumentParser()
parser.parse_args()

In [ ]:
sys.argv = ['script.py', '-c']
parser = argparse.ArgumentParser()
parser.parse_args()

- Argumenty pozycyjne są rozróżniane na podstawie miejsca w linii komend, gdzie występują
- Argumenty opcjonalne (flagi) są rozróżniane na podstawie nazwy lub jej jednoznacznego prefixu
- Jedne i drugie dodajemy wywołując komendę add_argument i przekazując jej nazwę argumentu
- Dostęp do pobranych parametrów odbywa się poprzez obiekt Namespace, zwrócony przez metodę parse_args
    - zwrócony obiekt ma pola, których nazwy odpowiadają pierwszej długiej nazwie (lub pierwsze w ogóle, jesli nie ma długiej - dotyczy argumentów opcjonalnych)

In [ ]:
sys.argv = ['script.py', 'file.txt']
parser = argparse.ArgumentParser()
parser.add_argument("input_file")
args = parser.parse_args()
print(args.input_file)

Parametr `nargs` oznacza liczbę argumentów danego typu. Może przyjmować następujące wartości:

    - liczbę - program wymaga dokładnie takiej liczby parametrów
    - "?" - parametr jest opcjonalny (może wystąpić raz lub nie wystąpić)
    - "*" - parametr może wystąpić dowolną liczbę razy, lub nie wystąpić
    - "+" - parametr może wystąpić dowolną liczbę razy, ale minimum jeden
    - argparse.REMAINDER - parametr będzie zawierał wszystko "zbędne" z linii komend (np. celem przekazania do innego programu); musi to być ostatni parametr dodany do parsera

Domyślnie równe 1

Flaga nie jest parametrem

In [ ]:
sys.argv = ['script.py', 'file.txt', 'file1.txt', 'file2.txt']
parser = argparse.ArgumentParser()
parser.add_argument("input_files", nargs="+", help="Tekst pomocy")
parser.add_argument("output_file")
args = parser.parse_args()
print(args.input_files, args.output_file)

In [ ]:
sys.argv = ['script.py', '-h']
parser.parse_args()

Parametr `action` oznacza akcję do wykonania:

    - "store" - zapisz przekazaną wartość (domyślnie)
    - "store_const", "store_true", "store_false" - zapisz stałą wartość (dla store_const należy dodać parametr const, okreslający wartość do zapisania)
    - "append" - dopisz wartość do listy
    - "count" - zlicz wystąpienia

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-v", "--verbose", action='store_true')

sys.argv = ['script.py', '-v']
print(parser.parse_args())

sys.argv = ['script.py']
print(parser.parse_args())

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-v", "-V", "--verbose", action="count", help="Makes output of program more verbose")

sys.argv = ['script.py','-vv','--ver']
print(parser.parse_args())

print("---")

sys.argv = ['script.py','-h']
print(parser.parse_args())

Parametr `default`

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output_file", default="a.out")

sys.argv = ['script.py', '-o', 'output.file']
print(parser.parse_args())
print("---")

sys.argv = ['script.py']
print(parser.parse_args())
print("---")

sys.argv = ['script.py', '-o']
print(parser.parse_args())

`type` określa typ argumentu (albo argumentów). Najczęściej używamy `int` albo `float`, ale możemy definiować własne.

Właściwie możemy użyć też `list`, `tuple` itp. ale jest to mało użyteczne (jak działa konwersja stringa na krotkę?).

Może to być też nazwa klasy, pod warunkiem, że konstruktor przyjmuje jednego stringa.

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-d")

sys.argv = ['script.py','-d', '3']
args = parser.parse_args() 
print(args)
print(type(args.d))

parser = argparse.ArgumentParser()
parser.add_argument("-d", type=int)

sys.argv = ['script.py','-d', '3']
args = parser.parse_args() 
print(args)
print(type(args.d))

sys.argv = ['script.py','-d', '3.5']
print(parser.parse_args())

In [ ]:
def filename(arg):
    try:
        with open(arg, "r"):
            pass
    except OSError:
        raise argparse.ArgumentTypeError("File {!r} cannot be opened".format(arg))
    return arg
 
parser = argparse.ArgumentParser()
parser.add_argument("infile", type=filename)
print(parser.parse_args(['no_such_file.txt']))

In [ ]:
def filename(arg):
    with open(arg, "r"):
        return arg
    
parser = argparse.ArgumentParser()
parser.add_argument("infile", type=filename)
print(parser.parse_args(['no_such_file.txt']))

Inne opcjonalne parametry dla `add_argument`
- `metavar` - alternatywna nazwa argumentu do wyświetlenia w tekście pomocy, np. do tłumaczenia na inny język (w Jupyter notebook'u nie działa)
- `choices` - lista wartości jakie może przyjmować argument

`add_mutually_exclusive_group` - tworzy grupę parametrów, które nie mogą współwystępować

In [ ]:
import sys
import argparse

parser = argparse.ArgumentParser()
group = parser.add_mutually_exclusive_group()
group.add_argument("-v", "--verbose", action="store_true")
group.add_argument("-q", "--quiet", action="store_true")
print(parser.parse_args([])) # można przekazać listę parametrów do funkcji parse_args
print(parser.parse_args(['-q']))
print(parser.parse_args(['-q','-v']))

### Tap (Typed Argument Parser)

https://pypi.org/project/typed-argument-parser/

Można instalować jako typed-argument-parser, **nie tap**

In [ ]:
from tap import Tap as TypedArgumentParser

class MyArgs(TypedArgumentParser):
    a: float = 1  # acceleration
    pi: int = 3.14  # what value of pi should we use
    input_filename: str  # file to read
        
args = MyArgs().parse_args(["--input_filename", "log.log"])
print(args)

### Click

In [ ]:
import sys

sys.argv = "main.py --help".split()

# źródło: https://click.palletsprojects.com/en/7.x/

import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name',
              help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello %s!' % name)

if __name__ == '__main__':
    hello()

https://towardsdatascience.com/how-to-write-python-command-line-interfaces-like-a-pro-f782450caf0d